In [1]:
import numpy as np 
import pandas as pd

# clean for state

Load in the data frame. Get lists of column labels.

In [3]:
all_data = pd.read_csv("Data_Files/state_housing_zillow.csv")
col_labels = all_data.columns.tolist()
timestamps = col_labels[5:]

Tranpose the dataset so that it is indexed by time and each column represents a state.

In [4]:
price_by_state_series = pd.DataFrame(all_data[timestamps].values.T, columns=all_data['RegionName'], index=timestamps)
print(price_by_state_series)

RegionName     California          Texas        Florida       New York  \
2000-01-31  186429.815199  110538.064827  105825.018501  149355.915577   
2000-02-29  187056.461453  110598.134067  106054.245342  149887.769350   
2000-03-31  187902.587345  110627.268589  106332.113966  150399.764377   
2000-04-30  189739.274366  110771.865807  106896.949408  151500.926494   
2000-05-31  191874.146347  110865.509233  107506.063565  152640.235655   
...                   ...            ...            ...            ...   
2024-09-30  768060.404666  299577.201315  389685.944614  470329.929340   
2024-10-31  769640.332853  299320.975181  388747.458761  472704.198362   
2024-11-30  771401.672972  298954.558970  387558.837887  474502.099921   
2024-12-31  773183.509222  298684.555574  386475.569667  476224.642087   
2025-01-31  773347.013436  298510.934462  385540.771612  477332.960271   

RegionName   Pennsylvania       Illinois           Ohio        Georgia  \
2000-01-31   96116.779767  125204.519

Create a dataframe of labels. Add a column to one-hot encode whether the original dataframe contained any NaN values for that state. Default to 0 (False).

In [5]:
state_labels = pd.DataFrame(all_data[col_labels[:5]].values, columns=col_labels[:5], index=all_data['RegionName'])
print(state_labels)
state_labels.drop(columns=['StateName'], inplace=True)
state_labels['LeadingNaN'] = 0
state_labels['ContainsNaN'] = 0

                     RegionID SizeRank            RegionName RegionType  \
RegionName                                                                
California                  9        0            California      state   
Texas                      54        1                 Texas      state   
Florida                    14        2               Florida      state   
New York                   43        3              New York      state   
Pennsylvania               47        4          Pennsylvania      state   
Illinois                   21        5              Illinois      state   
Ohio                       44        6                  Ohio      state   
Georgia                    16        7               Georgia      state   
North Carolina             36        8        North Carolina      state   
Michigan                   30        9              Michigan      state   
New Jersey                 40       10            New Jersey      state   
Virginia                 

Address NaN values. Replace any leading NaN values with the oldest recorded price measurement (backwards fill method). For intermediate NaN values, use take the average of the values before and after (interpolate). Update the ContainsNaN column of the labels dataframe.

In [6]:
# find columns with NaN values and update the column in labels
col_nans = price_by_state_series.isna().any()
print(np.sum(col_nans.values))
#print(col_nans)

# interpolate intermediate NaN values
for state in all_data['RegionName']:
    #print(col_nans[state])
    if col_nans[state]:
        all_nans = price_by_state_series[state].isna().sum()
        price_by_state_series[state].interpolate(method='linear', inplace=True)
        new_nans = price_by_state_series[state].isna().sum()
        state_labels.loc[state, 'ContainsNaN'] = all_nans - new_nans
        #if all_nans - new_nans != 0:
        print(state, all_nans - new_nans, all_nans, new_nans)   

print()
# backwards fill leading NaN values
col_nans = price_by_state_series.isna().any()
print(np.sum(col_nans.values))
for state in all_data['RegionName']:
    if col_nans[state]:
        print(state, price_by_state_series[state].isna().sum())
        state_labels.loc[state, 'LeadingNaN'] = price_by_state_series[state].isna().sum()
        price_by_state_series[state].fillna(method='bfill', inplace=True)


9
Arizona 1 1 0
New Mexico 0 27 27
Idaho 1 1 0
West Virginia 1 1 0
Montana 0 61 61
South Dakota 1 1 0
North Dakota 0 108 108
Alaska 1 1 0
Wyoming 0 27 27

4
New Mexico 27
Montana 61
North Dakota 108
Wyoming 27


In [7]:
col_nans = price_by_state_series.isnull().any()
print(col_nans)

RegionName
California              False
Texas                   False
Florida                 False
New York                False
Pennsylvania            False
Illinois                False
Ohio                    False
Georgia                 False
North Carolina          False
Michigan                False
New Jersey              False
Virginia                False
Washington              False
Arizona                 False
Massachusetts           False
Tennessee               False
Indiana                 False
Maryland                False
Missouri                False
Wisconsin               False
Colorado                False
Minnesota               False
South Carolina          False
Alabama                 False
Louisiana               False
Kentucky                False
Oregon                  False
Oklahoma                False
Connecticut             False
Utah                    False
Iowa                    False
Nevada                  False
Arkansas                False

Export cleaned datasets.

In [8]:
price_by_state_series.to_csv("Data_Files/price_by_state_cleaned.csv")
state_labels.to_csv("Data_Files/state_data_labels.csv")

# clean for county

Load in the data frame. Get lists of column labels.

In [334]:
all_data = pd.read_csv("Data_Files/county_housing_zillow.csv")
col_labels = all_data.columns.tolist()
timestamps = col_labels[9:]

Tranpose the dataset so that it is indexed by time and each column represents a county -- index by RegionID because county names are not unique!

In [336]:
price_by_county_series = pd.DataFrame(all_data[timestamps].values.T, columns=all_data['RegionID'], index=timestamps)
print(price_by_county_series.shape)

(300, 3073)


In [337]:
price_by_county_series

RegionID,3101,139,1090,2402,2841,1286,581,2964,978,2832,...,2088,1097,2518,2581,240,846,1648,1432,2794,2825
2000-01-31,205554.642143,146088.131319,109502.880748,143224.384045,213034.659393,2.483530e+05,200639.071495,119668.621108,93990.540943,149547.902766,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-29,205787.076303,146058.811433,109474.767351,143516.508250,213878.120425,2.505451e+05,201602.627381,120083.060160,94042.350458,149886.943085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-31,206645.170206,146299.372635,109313.922526,143902.521074,214900.510511,2.521021e+05,202654.141877,120436.828134,94119.405750,150300.974680,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-30,208344.164074,146991.131284,109247.781175,144696.080612,217215.437980,2.552860e+05,204936.099271,121153.716601,94297.411981,151248.156464,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-31,210504.871387,147876.373157,109221.904554,145570.897200,219715.973772,2.580317e+05,207366.081909,121787.881479,94530.243952,152486.072483,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-31,857411.008912,302846.384103,282907.172194,468743.383602,935687.520148,1.123102e+06,846570.558934,531887.523252,312967.856389,604212.620117,...,230463.934800,123386.090629,528411.317244,185303.801165,310436.823833,336736.175154,315766.403474,164047.103362,318725.125071,158653.532384
2024-09-30,863957.219674,303511.080209,282692.273148,467798.713506,935242.345948,1.130097e+06,849704.713103,533291.737261,312401.969738,605260.053955,...,233933.291898,123872.022476,530029.921353,185987.546582,315129.516712,346631.949352,318169.837605,164787.407684,329326.702388,161280.339942
2024-10-31,869036.446028,303864.098187,282325.752609,467070.117625,934533.913892,1.136611e+06,851903.458103,533542.030514,311843.578865,605860.776709,...,238367.449855,124136.559601,532923.729681,187909.810346,320006.941594,354998.496627,322748.075604,166054.747665,339183.091984,165289.537724
2024-11-30,873116.490139,304254.182907,281874.816774,466405.864875,934725.657369,1.143075e+06,854880.300260,532797.604892,311106.260896,606543.758767,...,241060.345103,123836.154642,531769.821932,188521.945064,321877.019649,358646.215997,326448.913102,166257.891113,344650.536347,168560.091132


Create a dataframe of labels. Add a column to one-hot encode whether the original dataframe contained any NaN values for that state. Default to 0 (False).

In [338]:
county_labels = pd.DataFrame(all_data[col_labels[:9]].values, columns=col_labels[:9], index=all_data['RegionID'])
county_labels['LeadingNaN'] = 0
county_labels['ContainsNaN'] = 0

Address NaN values. Replace any leading NaN values with the oldest recorded price measurement (backwards fill method). For intermediate NaN values, use take the average of the values before and after (interpolate). Update the ContainsNaN column of the labels dataframe.

In [ ]:
# find columns with NaN values and update the column in labels
col_nans = price_by_county_series.isna().any()

# interpolate intermediate NaN values
#print(col_nans)
for county in all_data['RegionID']:
    # print(county)
    # print(col_nans[county])
    if col_nans[county]:
        all_nans = price_by_county_series[county].isna().sum()
        price_by_county_series[county].interpolate(method='linear', inplace=True)
        new_nans = price_by_county_series[county].isna().sum()
        county_labels.loc[county, 'ContainsNaN'] = all_nans - new_nans
        #if all_nans - new_nans != 0:

# backwards fill leading NaN values
col_nans = price_by_county_series.isna().any()
# print(np.sum(col_nans.values))
for county in all_data['RegionID']:
    if col_nans[county]:
        # print(county, price_by_county_series[county].isna().sum())
        county_labels.loc[county, 'LeadingNaN'] = price_by_county_series[county].isna().sum()
        price_by_county_series[county].fillna(method='bfill', inplace=True)

2402 1 1 0
581 2 2 0
978 1 1 0
1347 1 1 0
791 1 1 0
2452 3 3 0
401 1 1 0
3159 1 1 0
281 1 1 0
1165 0 42 42
1388 1 1 0
1322 1 1 0
1776 4 88 84
2322 1 1 0
1106 1 1 0
2889 0 29 29
2732 0 20 20
322 0 4 4
2441 1 1 0
2983 1 1 0
1822 1 15 14
3081 2 93 91
2776 0 31 31
2980 1 1 0
2537 0 27 27
2934 0 29 29
221 0 44 44
66 2 14 12
3266 0 27 27
685 2 2 0
1585 0 12 12
2507 0 28 28
1721 1 1 0
3223 0 12 12
1953 5 5 0
1894 5 12 7
367 0 43 43
1712 1 1 0
1957 3 3 0
1638 1 1 0
1146 1 1 0
1161 0 48 48
98 0 105 105
3158 2 2 0
470 1 1 0
3200 0 60 60
1272 0 58 58
1670 1 1 0
2702 0 12 12
563 0 22 22
2318 0 41 41
3141 0 38 38
999 0 13 13
3112 0 44 44
1891 1 1 0
1057 0 91 91
2255 3 3 0
1622 1 1 0
1300 4 4 0
1110 0 36 36
1196 1 13 12
344 0 27 27
2849 0 91 91
410 1 22 21
2156 0 24 24
328 0 108 108
3225 0 44 44
2188 2 2 0
802 1 1 0
3122 0 45 45
2429 0 108 108
2754 0 60 60
1503 0 4 4
126 0 12 12
3254 2 2 0
2770 0 44 44
423 0 92 92
553 1 1 0
770 0 24 24
178 0 84 84
225 4 4 0
725 0 108 108
890 0 108 108
562 0 105 105


Export cleaned datasets.

In [343]:
price_by_county_series.to_csv("Data_Files/price_by_county_cleaned.csv")
county_labels.to_csv("Data_Files/county_data_labels.csv")